In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load MobileNetV2 pre-trained on ImageNet from TensorFlow Hub
base_model = hub.load("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4")

# Freeze the base model layers
base_model.trainable = False

# Create a new model by adding a classification head
model = models.Sequential([
    base_model,
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')  # Binary classification (food or non-food)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Create data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/path/to/food_dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/path/to/food_dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Train the model
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

# Save the model
model.save('food_detection_model.h5')

# Optionally, fine-tune the model on your specific food dataset by unfreezing some layers and retraining
# Set more layers as trainable
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Continue training the model
history_fine_tune = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

# Save the fine-tuned model
model.save('food_detection_model_fine_tuned.h5')
